In [2]:
import pandas as pd
import os
import codecs
import re
import numpy as np 
import json
import pickle
import matplotlib.pyplot as plt

In [40]:
###Read surprisal and audibility and clean

surprisal_new = pd.read_csv("./trf_input/word_surprisal_gpt_target.csv")

surprisal_new['words'] = surprisal_new['words'].str.replace(",", "")
surprisal_new['words'] = surprisal_new['words'].str.replace(".", "")
surprisal_new['words'] = surprisal_new['words'].str.replace(":", "")
surprisal_new['words'] = surprisal_new['words'].str.replace('"', "")
surprisal_new['words'] = surprisal_new['words'].str.replace('!', "")

audibility = pd.read_csv("./trf_input/audibility_words_final.csv")

audibility['word'] = audibility['word'].str.replace(",", "")
audibility['word'] = audibility['word'].str.replace(".", "")
audibility['word'] = audibility['word'].str.replace(":", "")
audibility['word'] = audibility['word'].str.replace('"', "")
audibility['word'] = audibility['word'].str.replace('!', "")


surprisal_phone = pd.read_csv("./trf_input/phone_surprisal_entropy_final.csv")

word_frequency = pd.read_csv("./trf_input/word_frequency_target_lgsubtlex.csv")
word_frequency['frequency']=np.where(word_frequency['word_order'] < 2, np.nan, word_frequency['frequency'])


surprisal_new['surprisal']=np.where(surprisal_new['word_order'] < 2, np.nan, surprisal_new['surprisal'])
surprisal_new['entropy']=np.where(surprisal_new['word_order'] < 2, np.nan, surprisal_new['entropy'])


#Phone audibility (only for timing info)
audibility_phone = pd.read_csv("./trf_input/audibility_phones_final.csv")

audibility_phone['words'] = audibility_phone['words'].str.replace(",", "")
audibility_phone['words'] = audibility_phone['words'].str.replace(".", "")
audibility_phone['words'] = audibility_phone['words'].str.replace(":", "")
audibility_phone['words'] = audibility_phone['words'].str.replace('"', "")
audibility_phone['words'] = audibility_phone['words'].str.replace('!', "")



In [41]:
## Load subject responses
sent = []
names = []
part = []
sub = []

outpath = "/data/pt_02917/transcriptions_final/"

for path, subdirs, files in os.walk(outpath):
    for name in files:
        if name.endswith(".lab") and name.startswith("sub") or name.startswith("41") or name.startswith("67"):

                #load all audios
            infile = os.path.join(path, name)

            f = codecs.open(infile, "r", "utf-8-sig")
            lines = f.read()

            lines = lines.replace("ï»¿ ", "")
            if name.startswith("41"):
                sub.append("sub41")
            elif name.startswith("67"):
                sub.append("sub67")
            else:
                sub.append(name[0:5])


            sent.append(lines)
            names.append(name)

sub_result = pd.DataFrame(zip(names, sent), columns = ['name', 'resp'])           
sub_result['target_id'] = sub_result['name'].str[-7:-4]
sub_result['subject'] = sub

sub_result['target_id'] = pd.to_numeric(sub_result['target_id'])



In [42]:
##Load subject experiment files
import scipy.stats as stats
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()



##For all pilot subjects
subs = ['sub05', 'sub06', 'sub07', 'sub08', 'sub09', 'sub10', 'sub11', 'sub12', 'sub13', 'sub14', 'sub15', 'sub16', 
        'sub18', 'sub19', 'sub20', 'sub21', 'sub22', 'sub23', 'sub24', 'sub25', 'sub26', 'sub27', 'sub28', 'sub29', 
       'sub30', 'sub31', 'sub32', 'sub33', 'sub34', 'sub35', 'sub36', 'sub37', 'sub38', 'sub39', 'sub40', 'sub41', 
       'sub42', 'sub43', 'sub44', 'sub45', 'sub46', 'sub47', 'sub48', 'sub49', 'sub50', 'sub51', 'sub52', 'sub53', 
       'sub54', 'sub55', 'sub56', 'sub57', 'sub58', 'sub59', 'sub60', 'sub61', 'sub62', 'sub63', 'sub64', 'sub65', 'sub66', 
       'sub67', 'sub68', 'sub69', 'sub70',  'sub72']

#subs = ['sub05']
subs_fulldf = []
for sub in subs: 
    subnum = sub[3:]
    subnum = int(subnum)
    if subnum != 69:
        path_sub = "/data/pt_02917/data/{s}/main_exp/{s}_mainexp.csv".format(s = sub)

    elif subnum == 69:
        path_sub = "/data/pt_02917/data/sub69/main_exp/sub69_complete.csv"

    df = pd.read_csv(path_sub, sep = ";")

    if subnum == 41:
        df['subject'] = "sub41"
    elif subnum == 67:
        df['subject'] = "sub67"
    

    ##Create one row for every word 
    df = df.assign(sentence=df['sentence'].str.split(' ')).explode('sentence')
    #Create a column for the word order (important for merging) 
    w_order = []
    for sentence_id in df['target'].unique():
        sent_w = df[df['target']== sentence_id]
        wo = 1
        
        for i,r in sent_w.iterrows():
            w_order.append(wo)
            wo = wo + 1
    
    df['word_order'] = w_order
    ##Make sure that all letters are correctly represented
    df['sentence'] = df['sentence'].str.replace("Ãľ", "Ü")
    df['sentence'] = df['sentence'].str.replace("Ã¼", "ü")
    df['sentence'] = df['sentence'].str.replace("Ã¶", "ö")
    df['sentence'] = df['sentence'].str.replace("Ã¤", "ä")
    df['sentence'] = df['sentence'].str.replace("ÃŁ", "ß")
    df['sentence'] = df['sentence'].str.replace("ÃĦ", "Ä")
    df['sentence'] = df['sentence'].str.replace("Ãĸ", "Ö")
    #Remove Punctuation
    df['sentence'] = df['sentence'].str.replace(",", "")
    df['sentence'] = df['sentence'].str.replace(".", "")
    df['sentence'] = df['sentence'].str.replace(":", "")
    df['sentence'] = df['sentence'].str.replace('"', "")
    df['sentence'] = df['sentence'].str.replace('!', "")
    
   # df['sentence'] = df['sentence'].str.replace('[^\w\s]','', regex=True)


    #For merging
    df['target_id'] = df['target'].str[0:3]
    df['target_id'] = pd.to_numeric(df['target_id'])


    df_full = pd.merge(df, surprisal_new, left_on = [ 'target_id', 'word_order'], right_on = [ 'id', 'word_order'], suffixes=('', '_y'))

    
    df_full.drop(['id', 'Unnamed: 0', 'words'], axis=1, inplace=True)
    df_full['words'] = df_full['sentence']
    df_full.drop(['sentence'], axis=1, inplace=True)
    

    df_full1 = pd.merge(df_full, audibility, left_on = [ 'target_id', 'word_order'], right_on = ['ids',  'word_order'])
    df_full1.drop(['Unnamed: 0', 'word', 'ids', 'filename'], axis=1, inplace=True)


    df_full2 = pd.merge(df_full1, word_frequency, left_on = ['target_id', 'word_order'], right_on = ['id',  'word_order'])
    ##Checking accuracy = Which words from the actual sentences are in the response

    acc = []
    misheard = []
    start_time = []
    end_time = []

    for target_id in df_full2['target_id'].unique():
        sent_df = df_full2.loc[df_full1['target_id'] == target_id]
        sub = sent_df.iloc[0,0]
    
        
        sub_responses = sub_result[sub_result['subject'] == sub]
        trial_resp = sub_responses[sub_responses['target_id'] == target_id]

        
        last_double_word = []

        if len(trial_resp) > 0:
            
            for i1,r1 in trial_resp.iterrows():
                resp = r1['resp']
        
        else:
            resp = ""

        resp = resp.lower()

        if resp == "nicht verstanden" or resp == "nichts verstanden":
            resp = ""


        resp = resp.replace('portraits', 'portäts')
        resp = resp.replace('brötchen backen', 'brötchenbacken')
        resp = resp.replace('skilaufen', 'ski laufen')
        resp = resp.replace('lös', 'löst')
        resp = resp.replace('einst weilen', 'einstweilen')
        resp = resp.replace('asthma-anfälle', 'asthmaanfälle')
        resp = resp.replace('12', 'zwölf')
        resp = resp.replace('umgehungsstrasse', 'umgehungsstraße')
        resp = resp.replace('30', 'dreißig')
        resp = resp.replace('heuaufen', 'heuhaufen')
        resp = resp.replace('8', 'acht')
        resp = resp.replace('2', 'zwei')
        resp = resp.replace('5', 'fünf')
        resp = resp.replace('flug', 'pflug')
        resp = resp.replace('1000', 'tausend')

        resp = resp.replace('gibt es', 'gibts')
        resp = resp.replace('über dem', 'überm')
        resp = resp.replace('Rauhreif', 'Raureif')
        
        resp = resp.replace('gewehr', 'gewähr')
        resp = resp.replace('oberleutnants', 'oberstleutnants')
        resp = resp.replace('bergs', 'berges')
        resp = resp.replace('bergesteiger', 'bergsteiger')

        resp = re.sub(r'[^\w\s]','', resp) 
        resp_w = resp.split(" ")

        
        
    
        for i,r in sent_df.iterrows():
            r['words'] = r['words'].lower()
            r['words'] = re.sub(r'[^\w\s]','', r['words']) 
    

            acc_t = any(r['words'] == w for w in resp_w)

            
            acc.append(acc_t)
       


    df_full2['acc'] = acc

    df_full2['surprisal']=np.where(df_full2['word_order'] < 2, np.nan, df_full2['surprisal'])
    df_full2['frequency']=np.where(df_full2['word_order'] < 2, np.nan, df_full2['frequency'])
    df_full2['entropy']=np.where(df_full2['word_order'] < 2, np.nan, df_full2['entropy'])

    df_phone = pd.merge(df_full2, audibility_phone, left_on = ['words', 'target_id', 'word_order'], right_on = ['words', 'ids', 'wordnum'])
        
    df_phone['audibility_word'] = df_phone['audibility_x']
    df_phone['audibility_phone'] = df_phone['audibility_y']

    df_phone.drop([ 'ids',  'filename', 'audibility_x', 'audibility_y'], axis=1, inplace=True)

    df_phone['words'] = df_phone['words'].str.lower()
   
    df_phone['phone_num']=df_phone.groupby(['words', 'target_id', 'word_order']).cumcount()+1

    df_full = pd.merge(df_phone, surprisal_phone, left_on = ['phone', 'words','target_id', 'phone_num'], right_on = ['phone','word', 'target_id', 'phone_num'])

    ##For saving (behav analysis)
    subs_fulldf.append(df_full2)


    ## TRF Feature creation (upsampling) ######
    ###Standardize the features


    
    df_full['phone_surprisal'] = min_max_scaler.fit_transform(df_full[['phone_surprisal']])
    df_full['phone_entropy'] = min_max_scaler.fit_transform(df_full[['phone_entropy']])
    df_full['surprisal'] = min_max_scaler.fit_transform(df_full[['surprisal']])    
    df_full['frequency'] = min_max_scaler.fit_transform(df_full[['frequency']])
    df_full['entropy'] = min_max_scaler.fit_transform(df_full[['entropy']])

    df_full['surprisal']=np.where(df_full['wordnum'] < 2, np.nan, df_full['surprisal'])
    df_full['frequency']=np.where(df_full['wordnum'] < 2, np.nan, df_full['frequency'])
    df_full['entropy']=np.where(df_full['wordnum'] < 2, np.nan, df_full['entropy'])

    ###Correct and incorrect ids
    id_cor = []
    id_f = []
    idb = []


    df_full_nb = df_full.copy()
   





df_full_con = pd.concat(subs_fulldf)
df_full_con.to_csv("../statistics/behavdat.csv", sep = ";")


In [43]:
feats = ['surprisal', 'frequency','entropy', 'phone_surprisal', 'phone_entropy', 'word_onset', 'phone_onset']

for feat in feats:
    upsamp = []

        
    for t,dfsent in df_full.groupby('target', sort = False):
        
        # Parameters
        sampling_rate = 128  # Hz
        duration = dfsent['end_s_x'].max() # add 1 sec buffer (or use known total duration)
        n_samples = int(np.ceil(duration * sampling_rate))
        
        # Initialize array with zeros
        signal = np.zeros(n_samples)

        if 'phone' in feat:
            onset_samples = (dfsent['start_s_y'] * sampling_rate).round().astype(int)
        else:
            # Convert onset times to sample indices
            onset_samples = (dfsent['start_s_x'] * sampling_rate).round().astype(int)

     
        
        if 'onset' in feat:
            signal[onset_samples] = 1
        else:
        
            # Insert values into the signal at the onset sample indices
            signal[onset_samples] = dfsent[feat].values
        signal = np.nan_to_num(signal)
    
        upsamp.append(signal)

        

        

        with open('./trf_input/{f}_target_final_yo.pickle'.format(f = feat), 'wb') as o:
            pickle.dump(upsamp, o)
    
    

In [44]:
#Save stimulus numbers
stimnum = df_full['target_id'].unique().tolist()
with open('./trf_input/stimnum_target.pickle', 'wb') as o:
    pickle.dump(stimnum, o)

## Distractor

In [49]:
import pandas as pd

word_surprisal = pd.read_csv("./trf_input/word_surprisal_gpt_dis.csv")
#
phone_surprisal = pd.read_csv("./trf_input/phone_surprisal_entropy_final_dis.csv")

directory = './word_seg_distractors/'

sent_id = []
words = []
start = []
end = []
word_order = []

for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            idx = file[:-4]
            p = os.path.join(directory, file)
            f=pd.read_csv(p, sep = ";")
            
            ##Remove silence in the beginning and end
            f = f.drop(f.loc[(f.MAU == "(...)")].index)
            
            f = f.drop(f.loc[(f.MAU.isnull())].index)

            f = f.drop(f.loc[f.TOKEN == -1].index)
    
            words_sent = list(f['TOKEN'].unique())
            wo = 1
            
            for w in words_sent:
                f_w = f.loc[f['TOKEN'] == w]
                word_start = f_w.iloc[0,0]
                word_end = f_w.iloc[-1,0] + f_w.iloc[-1,1]
                word = f_w.iloc[0,5]
    
                word_order.append(wo)
                wo = wo + 1
                
                sent_id.append(idx)
                #print(idx)
                words.append(word)
                start.append(word_start)
                end.append(word_end)

sent_words = pd.DataFrame(zip(sent_id, words, start, end, word_order), columns = ['id', 'word', 'start', 'end', 'word_order'])

##Calculate seconds
sent_words['start_s'] = sent_words['start']/44100
sent_words['end_s'] = sent_words['end']/44100


merge1 = pd.merge(sent_words, word_surprisal, on = ['id', 'word_order'])
merge1.drop(['words', 'Unnamed: 0'], axis = 1, inplace = True)



targets_complete = pd.read_csv('./target_distractor_final_final.csv', sep = ";")

directory = './word_seg_distractors/'


sent_id = []
words = []
phones = []
start = []
end = []
word_order = []
wordnum = []
syl_nums = []
syls = []

for root,dirs,files in os.walk(directory):
    for file in files:
        if file[:7] in list(targets_complete['distractor'].str[:7]):

            p = os.path.join(directory, file)
            if p.endswith(".csv"):
                f=pd.read_csv(p, sep = ";")
            
                ##Remove silence in the beginning and end
                f = f.drop(f.loc[(f.MAU == "(...)")].index)
                f = f.drop(f.loc[f.TOKEN == -1].index)
                f = f.drop(f.loc[(f.MAU.isnull())].index)

                f = f.reset_index(drop=True)
                wo = 1
        
                syl_add = ""
                syl_num = 1
                syl_count = 1
                
                for i,p1 in f.iterrows():
                    f_w = p1
                    phone_start = p1.iloc[0]
                    phone_end = p1.iloc[0] + p1.iloc[1]
                    phone = p1.iloc[3]
                    word = p1.iloc[5]
                    numw = int(p1.iloc[2])+1
        
                    word_order.append(wo)
                    wo = wo + 1
        
    
        
                    
                    sent_id.append(file[:7])
                    phones.append(phone)
                    words.append(word)
                    wordnum.append(numw)
        
                    start.append(phone_start)
                    end.append(phone_end)
                


sent_words = pd.DataFrame(zip(sent_id, words, phones, start, end, word_order, wordnum), columns = ['id', 'words', 'phone', 'start', 'end', 'phone_order', 'wordnum'])

##Calculate seconds
sent_words['start_s'] = sent_words['start']/44100
sent_words['end_s'] = sent_words['end']/44100

sent_words['phone_num'] = sent_words.groupby(['id', 'words', 'wordnum']).cumcount()+1


merge2 = pd.merge(sent_words, phone_surprisal, left_on = ['id', 'wordnum', 'phone_num'], right_on = ['id', 'word_order', 'phone_num'])
merge2.drop(['words', 'Unnamed: 0',  'wordnum', 'phone_order'], axis = 1, inplace = True)


merge_surpdis = pd.merge(merge1, merge2, on = ['id', 'word_order'])

freq_dis = pd.read_csv("./trf_input/word_frequency_dis_lgsubtlex.csv")


merge_surpdis_freq = pd.merge(merge_surpdis, freq_dis, on = ['id', 'word_order'])

merge_surpdis_freq['surprisal']=np.where(merge_surpdis_freq['word_order'] == 1, np.nan, merge_surpdis_freq['surprisal'])
merge_surpdis_freq['frequency']=np.where(merge_surpdis_freq['word_order'] == 1, np.nan, merge_surpdis_freq['frequency'])
merge_surpdis_freq['entropy']=np.where(merge_surpdis_freq['word_order'] == 1, np.nan, merge_surpdis_freq['entropy'])

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()


import scipy.stats as stats

merge_surpdis_freq['surprisal'] = min_max_scaler.fit_transform(merge_surpdis_freq[['surprisal']])
merge_surpdis_freq['phone_surprisal'] = min_max_scaler.fit_transform(merge_surpdis_freq[['phone_surprisal']])
merge_surpdis_freq['phone_entropy'] = min_max_scaler.fit_transform(merge_surpdis_freq[['phone_entropy']])
merge_surpdis_freq['frequency'] = min_max_scaler.fit_transform(merge_surpdis_freq[['frequency']])
merge_surpdis_freq['entropy']=min_max_scaler.fit_transform(merge_surpdis_freq[['entropy']])


In [50]:
feats = ['surprisal', 'frequency','entropy', 'phone_surprisal', 'phone_entropy', 'word_onset', 'phone_onset']

for feat in feats:
    upsamp = []

        
    for t,dfsent in merge_surpdis_freq.groupby('id', sort = False):
        
        # Parameters
        sampling_rate = 128  # Hz
        duration = dfsent['end_s_y'].max()   
        n_samples = int(np.ceil(duration * sampling_rate))
        
        # Initialize array with zeros
        signal = np.zeros(n_samples)

        if 'phone' in feat:
            onset_samples = (dfsent['start_s_y'] * sampling_rate).round().astype(int)
        else:
            # Convert onset times to sample indices
            onset_samples = (dfsent['start_s_x'] * sampling_rate).round().astype(int)

        
        if 'onset' in feat:
            signal[onset_samples] = 1
        else:
        
            # Insert values into the signal at the onset sample indices
            signal[onset_samples] = dfsent[feat].values
        signal = np.nan_to_num(signal)
    
        upsamp.append(signal)

        

        

        with open('./trf_input/{f}_dis_final_yo.pickle'.format(f = feat), 'wb') as o:
            pickle.dump(upsamp, o)
    
      

In [51]:
#Save stimulus numbers
stimnum = merge_surpdis_freq['id'].unique().tolist()
with open('./trf_input/stimnum_dis.pickle', 'wb') as o:
    pickle.dump(stimnum, o)

## Envelopes, derivative, and f0

In [64]:
from scipy.signal import butter, filtfilt, find_peaks, resample
import librosa 
import naplib
import matplotlib.pyplot as plt
import logging
import numpy as np
import pandas as pd
import scipy
import pickle
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
logging.getLogger("naplib").setLevel(logging.FATAL)

sub='sub05'

#Load behavioral log file
path_sub = "/data/pt_02917/data/{s}/main_exp/{s}_mainexp.csv".format(s = sub)
df = pd.read_csv(path_sub, sep = ";")

target_path = "../behavior_stimuli/targets_final_norm/"
distractor_path = "../behavior_stimuli/distractors_new_clean/"

envelopes_target = []
rates_target = []
target_f0 = []

envelopes_dis = []
rates_dis = []
dis_f0 = []
stimnum = []

from scipy.stats import zscore
for i,r in df.iterrows():
    num = int(r['target'][0:3])
    stimnum.append(num)

    target, sr = librosa.load(target_path + r['target'], sr = 44100)
    distractor, sr = librosa.load(distractor_path + r['distractor'], sr = 44100)
    

   # zero pad the target
    zer = np.array([0]*22050)
    y1_sil = np.concatenate((zer,target))
    
    zer2 = np.array([0]*(len(distractor)-len(y1_sil)))
    target_pad = np.concatenate((y1_sil, zer2))

    #####Target    
    ###Calculate the spectrogram 
    spec= naplib.features.auditory_spectrogram(target_pad, sr)

    # Extract the envelope by taking the mean over frequencies
    env = np.mean(spec, axis=1)
    env = scipy.signal.resample(env, round(len(env)/125*128))    
   
    # Temporal derivative 
    rate = np.maximum(np.diff(env, prepend=env[0]), 0)
    rate = scipy.signal.resample(rate, len(env))

    #F0
    f0t,v,vp= librosa.pyin(target_pad,sr=sr,fmin=75, fmax=175)
    f0t_p = np.nan_to_num(f0t, nan = np.nanmean(f0t))
    f0t_p = scipy.signal.resample(f0t_p, len(env))

    env_target = min_max_scaler.fit_transform(env.reshape(-1,1))
    rate_target = min_max_scaler.fit_transform(rate.reshape(-1,1))

    



    ###Same for the distractor
    spec= naplib.features.auditory_spectrogram(distractor, sr)

    # Extract the envelope by taking the mean over frequencies
    env = np.mean(spec, axis=1)
    env = scipy.signal.resample(env, round(len(env)/125*128))    
  
    # Temporal derivative 
    rate = np.maximum(np.diff(env, prepend=env[0]), 0)
    rate = scipy.signal.resample(rate, len(env))

    ##F0 
    f0d,v,vp = librosa.pyin(distractor,sr=sr,fmin=120, fmax=300)
    f0d_p = np.nan_to_num(f0d, nan = np.nanmean(f0d))   
    f0d_p = scipy.signal.resample(f0d_p, len(env))


    env = min_max_scaler.fit_transform(env.reshape(-1,1))
    rate = min_max_scaler.fit_transform(rate.reshape(-1,1))


    zer = np.array([0]*64)
    env_target = env_target.reshape(env_target.shape[0])
    rate_target = rate_target.reshape(env_target.shape[0])
    
  
    
    rates_target.append(rate_target)
    envelopes_target.append(env_target)
    target_f0.append(f0t_p)

    
    rates_dis.append(rate)
    envelopes_dis.append(env)
    dis_f0.append(f0d_p)






        

with open("./trf_input/envelope_target.pickle".format(sub = sub), "wb") as output_file:
    pickle.dump(envelopes_target, output_file)

with open("./trf_input/envelope_dis.pickle".format(sub = sub), "wb") as output_file:
    pickle.dump(envelopes_dis, output_file)


with open("./trf_input/rate_target.pickle".format(sub = sub), "wb") as output_file:
    pickle.dump(rates_target, output_file)

with open("./trf_input/rate_dis.pickle".format(sub = sub), "wb") as output_file:
    pickle.dump(rates_dis, output_file)


with open("./trf_input/stimnum_env.pickle".format(sub = sub), "wb") as output_file:
    pickle.dump(stimnum, output_file)



   
##Standardize F0 
all_f0_t = np.concatenate(target_f0)
all_f0_d = np.concatenate(dis_f0)

all_f0 = np.concatenate([all_f0_t, all_f0_d])

mean_f0 = np.mean(all_f0)
std_f0 = np.std(all_f0)

f0_target_stan = []
f0_dis_stan = []
for i,f in enumerate(target_f0):
    target_f0_stan = (f-mean_f0)/std_f0
    dis_f0_stan = (dis_f0[i]-mean_f0)/std_f0
    f0_target_stan.append(target_f0_stan)
    f0_dis_stan.append(dis_f0_stan)
        


with open("./trf_input/f0_target.pickle", "wb") as output_file:
    pickle.dump(f0_target_stan, output_file)

with open("./trf_input/f0_dis.pickle", "wb") as output_file:
    pickle.dump(f0_dis_stan, output_file)


In [62]:
r

subject                                                            sub05
trialnum                                                             240
gender_follow                                                       male
target                                                           004.wav
distractor                                                   dis_070.wav
sentence                             Das kalte Licht leuchtet am Morgen.
distractor_sentence    Hinter den Augen verbirgt sich die Magie des U...
srt_db                                                        -13.661274
srt_snr                                                        -6.830637
db_cond                                                               -2
trial_start                                                  11920.20497
sound_stop                                                  11924.154364
resp                                                                   1
respt                                              

## Speaker features

In [60]:
rat = pd.read_csv("./ratings.csv")

vertrauen = rat.iloc[1]['vertrauen.response']
age = rat.iloc[1]['age.response']
gebildet = rat.iloc[1]['gebildet.response']

dominant = rat.iloc[1]['Dominant.response']
attraktiv = rat.iloc[1]['Attraktiv.response']
gesund=rat.iloc[1]['gesund.response']
professionell = rat.iloc[1]['professionell.response']

dict_rat = {'vertrauen':vertrauen, 'age':age,'gebildet':gebildet, 'dominant':dominant, 'attraktiv': attraktiv, 
            'gesund':gesund,'professionell':professionell}

In [61]:
feats = ['vertrauen', 'age','gebildet', 'dominant', 'attraktiv', 'gesund', 'professionell']

for feat in feats:
    upsamp = []

        
    for t,dfsent in df_full.groupby('target', sort = False):
        
        # Parameters
        sampling_rate = 128  # Hz
        duration = dfsent['end_s_y'].max()   
        n_samples = int(np.ceil(duration * sampling_rate))
        
        # Initialize array with zeros
        signal = np.zeros(n_samples)

        
        # Convert onset times to sample indices
        onset_samples = (dfsent['start_s_x'] * sampling_rate).round().astype(int)

    

        # Insert values into the signal at the onset sample indices
        signal[onset_samples] = dict_rat[feat]
        
        signal = np.nan_to_num(signal)
    
        upsamp.append(signal)

        

        

        with open('./trf_input/{f}_target.pickle'.format(f = feat), 'wb') as o:
            pickle.dump(upsamp, o)

In [58]:
vertrauen = rat.iloc[0]['vertrauen.response']
age = rat.iloc[0]['age.response']
gebildet = rat.iloc[0]['gebildet.response']

dominant = rat.iloc[0]['Dominant.response']
attraktiv = rat.iloc[0]['Attraktiv.response']
gesund = rat.iloc[0]['gesund.response']
professionell = rat.iloc[0]['professionell.response']

dict_rat = {'vertrauen':vertrauen, 'age':age,'gebildet':gebildet, 'dominant':dominant, 'attraktiv': attraktiv, 
            'gesund':gesund,'professionell':professionell}

In [59]:
feats = ['vertrauen', 'age','gebildet', 'dominant', 'attraktiv', 'gesund', 'professionell']

for feat in feats:
    upsamp = []

        
    for t,dfsent in merge_surpdis_freq.groupby('id', sort = False):
        
        # Parameters
        sampling_rate = 128  # Hz
        duration = dfsent['end_s_y'].max()   
        n_samples = int(np.ceil(duration * sampling_rate))
        
        # Initialize array with zeros
        signal = np.zeros(n_samples)

        
        # Convert onset times to sample indices
        onset_samples = (dfsent['start_s_x'] * sampling_rate).round().astype(int)

    

        # Insert values into the signal at the onset sample indices
        signal[onset_samples] = dict_rat[feat]
        
        signal = np.nan_to_num(signal)
    
        upsamp.append(signal)

        

        

        with open('./trf_input/{f}_dis.pickle'.format(f = feat), 'wb') as o:
            pickle.dump(upsamp, o)